In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from definitions import load_caracteristics_csv, load_users_csv, load_places_csv, load_vehicles_csv
from tueplots import bundles





In [ ]:
data_caracteristics = load_caracteristics_csv()
data_users = load_users_csv()
data_places = load_places_csv()
data_vehicles = load_vehicles_csv()

In [ ]:
lr_data_caracteristics = data_caracteristics[['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum',
                                             'agg', 'int', 'atm', 'col']]
#lum, hrmn
print(lr_data_caracteristics.head())
lr_data_places = data_places[['Num_Acc', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'surf',
                             'infra', 'situ']]
print(lr_data_places.head())

lr_data_users = data_users[['Num_Acc', 'catu', 'grav', 'sexe', 'an_nais', 'trajet',
                           'locp', 'actp', 'etatp']]
#an_nais in alter umrechnen
print(lr_data_users.head())

In [ ]:
users_caracteristics_data = pd.merge(lr_data_users,
                                    lr_data_caracteristics,
                                    how='left',
                                    on=['Num_Acc', 'Num_Acc'])
ucpd = pd.merge(users_caracteristics_data,
                                           lr_data_places,
                                           how='left',
                                           on=['Num_Acc', 'Num_Acc'])

def convert_lum(x):
    if x == 5:
        return 3
    if x == 3:
        return 5
    return x

def convert_hrmn(x):
    hours, minutes = divmod(x, 100)
    return hours * 60 + minutes


ucpd['lum'] = ucpd['lum'].map(lambda a: convert_lum(a))
ucpd['hrmn'] = ucpd['hrmn'].map(lambda a: convert_hrmn(a))
ucpd['an_nais'] = 2000 + ucpd['an'] - ucpd['an_nais']

ucpd[['lum', 'hrmn', 'an_nais']].head()
ucpd.dtypes

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

ucpd.dropna(inplace=True)

X = ucpd.loc[:, ucpd.columns!= 'grav']
X = X.loc[:, X.columns!='Num_Acc']
Y = ucpd[['grav']]


#for column in X.columns:
#    print(X[[column]].value_counts())

reg = LinearRegression().fit(X, Y)
reg.coef_

#1876005

In [ ]:
plt.barh(X.columns, np.reshape(reg.coef_, newshape=-1), height=0.9)
plt.savefig('..//Plots//Linear_Regression_grav.pdf')
plt.show()

In [ ]:
bike_lr_data_users = data_users[['Num_Acc', 'catu', 'grav', 'sexe', 'an_nais', 'trajet',
                           'locp', 'actp', 'etatp', 'num_veh']]
bike_lr_data_vehicles = data_vehicles[['Num_Acc', 'num_veh', 'catv']]


In [ ]:
bike_user_data = pd.merge(bike_lr_data_users,
                         bike_lr_data_vehicles,
                         how='left',
                         left_on=['Num_Acc', 'num_veh'],
                         right_on=['Num_Acc', 'num_veh'])

bike_user_data = bike_user_data.loc[bike_user_data['catv'] == 1]
bike_user_data = bike_user_data.loc[:, bike_user_data.columns != 'num_veh']

bike_users_caracteristics_data = pd.merge(bike_user_data,
                                    lr_data_caracteristics,
                                    how='left',
                                    on=['Num_Acc', 'Num_Acc'])

bucpd = pd.merge(bike_users_caracteristics_data,
                                           lr_data_places,
                                           how='left',
                                           on=['Num_Acc', 'Num_Acc'])

bucpd['lum'] = bucpd['lum'].map(lambda a: convert_lum(a))
bucpd['hrmn'] = bucpd['hrmn'].map(lambda a: convert_hrmn(a))
bucpd['an_nais'] = 2000 + bucpd['an'] - bucpd['an_nais']

bucpd.dropna(inplace=True)

In [ ]:
bike_X = bucpd.loc[:, bucpd.columns!= 'grav']
bike_X = bike_X.loc[:, bike_X.columns!='Num_Acc']
bike_X = bike_X.loc[:, bike_X.columns!='catv']
bike_Y = bucpd[['grav']]

bike_reg = LinearRegression().fit(bike_X, bike_Y)
bike_reg.coef_

In [ ]:
plt.barh(bike_X.columns, np.reshape(bike_reg.coef_, newshape=-1), height=0.9)
plt.savefig('..//Plots//Bike_Linear_Regression_grav.pdf')
plt.show()